## Preliminary preparations

In [18]:
from datetime import datetime
import os
from pathlib import Path

import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
pd.set_option('display.max_rows', 100)

In [19]:
def find_project_root() -> Path:
    current_dir = Path(os.getcwd())
    if (current_dir / "pyproject.toml").exists():
        return current_dir
    for parent in current_dir.parents:
        if (parent / "pyproject.toml").exists():
            return parent

## Preparing Data Source

In [21]:
data = []

with open("../logs/128G/transaction-only-128GB-2024-10-07-06-55-38-mainnet-beta.log", 'r') as file:
    for line in file:
        if "datapoint: replay-slot-stats" in line:

            timestamp_match = re.search(r"\[(.*?)Z", line)
            transactions_match = re.search(r"total_transactions=(\d+)i", line)
            execute_us_match = re.search(r"execute_us=(\d+)i", line)
            
            if timestamp_match and transactions_match and execute_us_match:
                timestamp = timestamp_match.group(1)
                total_transactions_executed = int(transactions_match.group(1))
                execute_us = int(execute_us_match.group(1))

                data.append([timestamp, total_transactions_executed, execute_us])

df = pd.DataFrame(data, columns=['timestamp', 'total_transactions_executed', 'execute_us'])
df

,timestamp,total_transactions_executed,execute_us
0,2024-10-07T09:32:51.345610800,1019,753235
1,2024-10-07T09:32:56.351082564,1072,587296
2,2024-10-07T09:32:57.397677861,2215,643038
3,2024-10-07T09:32:59.756236618,1684,667559
4,2024-10-07T09:33:01.285146452,1389,466778
...,...,...,...
8676,2024-10-07T12:08:12.617945334,1050,4712362
8677,2024-10-07T12:08:22.825841644,1153,10593918
8678,2024-10-07T12:08:24.258412322,1685,2843369
8679,2024-10-07T12:08:26.165700311,1039,1372793


In [22]:
def convert_to_unixtime(timestamp):
    base_time, microseconds = timestamp.split('.')
    microseconds = microseconds[:6]
    dt = datetime.strptime(base_time, '%Y-%m-%dT%H:%M:%S')
    return int(dt.timestamp() * 1_000_000) + int(microseconds)

In [23]:
df['end'] = df['timestamp'].apply(convert_to_unixtime)
df['start'] = df['end'] - df['execute_us']
df['tps'] = df['total_transactions_executed'] * 1000000 / df['execute_us']
df


,timestamp,total_transactions_executed,execute_us,end,start,tps
0,2024-10-07T09:32:51.345610800,1019,753235,1728318771345610,1728318770592375,1352.831454
1,2024-10-07T09:32:56.351082564,1072,587296,1728318776351082,1728318775763786,1825.314662
2,2024-10-07T09:32:57.397677861,2215,643038,1728318777397677,1728318776754639,3444.586479
3,2024-10-07T09:32:59.756236618,1684,667559,1728318779756236,1728318779088677,2522.623468
4,2024-10-07T09:33:01.285146452,1389,466778,1728318781285146,1728318780818368,2975.718650
...,...,...,...,...,...,...
8676,2024-10-07T12:08:12.617945334,1050,4712362,1728328092617945,1728328087905583,222.818196
8677,2024-10-07T12:08:22.825841644,1153,10593918,1728328102825841,1728328092231923,108.836032
8678,2024-10-07T12:08:24.258412322,1685,2843369,1728328104258412,1728328101415043,592.606869
8679,2024-10-07T12:08:26.165700311,1039,1372793,1728328106165700,1728328104792907,756.851179


In [24]:
start_time = df.start.min() // 1000000 * 1000000
end_time = (df.start.max() + 1000000) // 1000000 * 1000000
time_step = 100000 # 10,000 microsends = .1 second

time_series = pd.DataFrame({
    't': np.arange(start_time, end_time + time_step, time_step)
})

def aggregate_tps(row, df):
    mask = (df['start'] <= row['t']) & (df['end'] >= row['t'])
    return df.loc[mask, 'tps'].sum()

time_series['sum_tps'] = time_series.apply(aggregate_tps, axis=1, df=df)
# window=10 means creating a moving average with the past 1 second's data. If you change it to 100, it would be the past 10 seconds
time_series['moving_average'] = time_series['sum_tps'].rolling(window=50, min_periods=1).mean()
time_series.to_csv('tps_128gb.csv', index=False)


In [25]:
# Calculate the maximum TPS
max_tps = time_series['sum_tps'].max()

# Calculate the overall average TPS
average_tps = time_series['sum_tps'].mean()

# Print the results
print(f"Maximum TPS: {max_tps:,.2f}")
print(f"Average TPS: {average_tps:,.2f}")

# Convert the max TPS x-coordinate to the corresponding time
max_tps_time = time_series.loc[time_series['sum_tps'].idxmax(), 't']

# Calculate the total duration spent with 0 TPS
time_with_zero_tps = time_series[time_series['sum_tps'] == 0]
total_time_zero_tps = len(time_with_zero_tps) * time_step / 1_000_000  # Convert microseconds to seconds
total_time_zero_tps_hours = total_time_zero_tps / 3600  # Convert seconds to hours

print(f"Total time spent with 0 TPS: {total_time_zero_tps:.2f} seconds ({total_time_zero_tps_hours:.2f} hours)")

Maximum TPS: 27,603.43
Average TPS: 1,219.08
Total time spent with 0 TPS: 2325.30 seconds (0.65 hours)


In [26]:
# Display every 10 seconds
filtered_time_series = time_series[time_series['t'] % 10000000 == 0]
# from micro sec to sec
filtered_time_series['t'] = np.floor(filtered_time_series['t'] / 1000000)

filtered_time_series

,t,sum_tps,moving_average
0,1.728319e+09,0.000000,0.000000
100,1.728319e+09,0.000000,985.555423
200,1.728319e+09,2280.903225,45.618064
300,1.728319e+09,827.958922,202.574580
400,1.728319e+09,0.000000,370.789102
...,...,...,...
92900,1.728328e+09,1551.446873,919.152529
93000,1.728328e+09,202.732270,1647.554646
93100,1.728328e+09,1520.666665,1520.119754
93200,1.728328e+09,222.818196,735.820903


In [27]:
# Calculate time elapsed in seconds from the start of the log
start_time = filtered_time_series['t'].min()  # First timestamp
filtered_time_series['elapsed_time'] = filtered_time_series['t'] - start_time
filtered_time_series['elapsed_time_hours'] = filtered_time_series['elapsed_time'] / 3600

# Convert elapsed time to HH:MM:SS format
filtered_time_series['elapsed_time_formatted'] = filtered_time_series['elapsed_time'].apply(
    lambda x: f"{int(x // 3600):02}:{int((x % 3600) // 60):02}:{int(x % 60):02}"
)

In [29]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=filtered_time_series['elapsed_time_hours'], y=filtered_time_series['sum_tps'], mode='markers', name='TPS', marker=dict(size=2)))
fig.add_trace(go.Scatter(x=filtered_time_series['elapsed_time_hours'], y=filtered_time_series['moving_average'], mode='lines', name='TPS Moving Average(1sec)', line=dict(width=0.5)))

# Add horizontal lines for max and average TPS
fig.add_hline(y=average_tps, line_dash="dot", line_color="green", annotation_text=f"Average TPS: {average_tps:.2f}", annotation_position="bottom right")
fig.add_hline(y=max_tps, line_dash="dash", line_color="red", annotation_text=f"Max TPS: {max_tps:.2f}", annotation_position="top right")

fig.update_layout(
    title=f"TPS Over Time | Total Time Spent with 0 TPS: {total_time_zero_tps:.2f} seconds ({total_time_zero_tps_hours:.2f} hours)",
    xaxis_title="Time Elapsed (hours)",
    yaxis_title="TPS",
    xaxis=dict(tickmode='linear',
        dtick=1,
        tickformat=".1f"),
    yaxis=dict(range=[0, 15000])
)


fig.show()


## Notes
- If all data points are plotted on the graph, the points and lines become indistinguishable, so I plot them every 1 seconds.
- TPS values above 10000 are excluded as outliers.
- Upon reviewing tps_128gb.csv, it was found that there are quite a few areas where tps is 0. As a result, the moving average is also fluctuating.
